In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.merge_peaks.240624/pairs.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.merge_peaks.240624/matrix.atac.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.merge_peaks.240624/matrix.rna.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/2.Genome_wide_prediction/Signac/PBMC.240624/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [4]:
matrix.atac = readRDS(path.matrix.atac_count)

Import RNA matrix

In [5]:
matrix.rna_count = readRDS(path.matrix.rna_count)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Prepare background data

In [6]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 303411 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100028331-100029533      *
       [2]     chr1 100037542-100039303      *
       [3]     chr1 100044514-100044850      *
       [4]     chr1 100045353-100045836      *
       [5]     chr1 100046102-100047652      *
       ...      ...                 ...    ...
  [303407]     chrX     9988850-9989079      *
  [303408]     chrX       999238-999388      *
  [303409]     chrX       999443-999815      *
  [303410]     chrX     9995651-9996434      *
  [303411]     chrX     9997538-9997892      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [7]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100028331-100029533,89,64,91,70,91,72,17,92,77,57,⋯,58,57,80,84,143,43.64090,1203,chr1,892,0.85404616
chr1-100037542-100039303,107,80,105,61,88,164,136,121,94,158,⋯,97,64,106,109,102,58.34279,1762,chr1,20499,0.99189548
chr1-100044514-100044850,10,14,19,18,21,25,3,33,14,24,⋯,14,16,19,30,54,46.29080,337,chr1,89,0.36970973
chr1-100045353-100045836,30,25,27,50,32,11,8,31,23,12,⋯,28,48,34,28,71,35.12397,484,chr1,160,0.55428445
chr1-100046102-100047652,72,45,81,121,77,68,21,123,77,68,⋯,107,92,109,150,270,39.39394,1551,chr1,1812,0.91444608
chr1-100048639-100048789,11,10,4,9,11,7,1,19,5,6,⋯,7,7,16,13,18,41.72185,151,chr1,112,0.44754805
chr1-100048869-100049158,20,11,15,20,18,26,4,30,14,16,⋯,10,15,25,20,36,43.79310,290,chr1,165,0.56253399
chr1-100050949-100051121,25,10,15,5,13,12,8,8,11,12,⋯,10,6,7,10,5,52.02312,173,chr1,80,0.33199851
chr1-100051301-100052036,65,41,47,67,52,29,7,47,40,27,⋯,40,63,38,52,95,36.27717,736,chr1,458,0.76908220


Run Signac prediction

In [8]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [9]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 5543100 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100028331-100029533      * |         AGL
        [2]     chr1 100028331-100029533      * |      CDC14A
        [3]     chr1 100028331-100029533      * |         DBT
        [4]     chr1 100028331-100029533      * |        DPH5
        [5]     chr1 100028331-100029533      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [5543096]     chrX     9995651-9996434      * |        WWC3
  [5543097]     chrX     9997538-9997892      * |       CLCN4
  [5543098]     chrX     9997538-9997892      * |      GPR143
  [5543099]     chrX     9997538-9997892      * |       TBL1X
  [5543100]     chrX     9997538-9997892      * |        WWC3
                          PeakName               PairName    zscore    pvalue
                       <character>            <character>

In [10]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna,
                                 matrix.atac,
                                 meta.features,
                                 16)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

Time difference of 12.10942 hours

In [11]:
execution_time

Time difference of 12.10942 hours

Save results

In [12]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 5543100 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100028331-100029533      * |         AGL
        [2]     chr1 100037542-100039303      * |         AGL
        [3]     chr1 100044514-100044850      * |         AGL
        [4]     chr1 100045353-100045836      * |         AGL
        [5]     chr1 100046102-100047652      * |         AGL
        ...      ...                 ...    ... .         ...
  [5543096]     chrX   97563241-97564955      * |      DIAPH2
  [5543097]     chrX   97597778-97598326      * |      DIAPH2
  [5543098]     chrX   97615894-97616098      * |      DIAPH2
  [5543099]     chrX   97616988-97617525      * |      DIAPH2
  [5543100]     chrX   97622133-97622400      * |      DIAPH2
                          PeakName               PairName    zscore     pvalue
                       <character>            <character

In [13]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100028331,100029533,1203,*,AGL,chr1-100028331-100029533,chr1-100028331-100029533_AGL,0.48442360,0.3140426368,K562
chr1,100037542,100039303,1762,*,AGL,chr1-100037542-100039303,chr1-100037542-100039303_AGL,3.29474243,0.0004925596,K562
chr1,100044514,100044850,337,*,AGL,chr1-100044514-100044850,chr1-100044514-100044850_AGL,1.52685771,0.0633981984,K562
chr1,100045353,100045836,484,*,AGL,chr1-100045353-100045836,chr1-100045353-100045836_AGL,0.97106612,0.1657576757,K562
chr1,100046102,100047652,1551,*,AGL,chr1-100046102-100047652,chr1-100046102-100047652_AGL,-0.96382835,0.8324340059,K562
chr1,100048639,100048789,151,*,AGL,chr1-100048639-100048789,chr1-100048639-100048789_AGL,-0.18561559,0.5736268882,K562
chr1,100048869,100049158,290,*,AGL,chr1-100048869-100049158,chr1-100048869-100049158_AGL,-1.19591297,0.8841347389,K562
chr1,100050949,100051121,173,*,AGL,chr1-100050949-100051121,chr1-100050949-100051121_AGL,0.38112950,0.3515535779,K562
chr1,100051301,100052036,736,*,AGL,chr1-100051301-100052036,chr1-100051301-100052036_AGL,-0.97612379,0.8354984389,K562


In [14]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2                     Matrix_1.6-5                     
 [3] BSgenome.Hsapiens.UCSC.hg38_1.4.5 BSgenome_1.